# 1. SET UP, FULL EXAMPLE BASED ON df_asc_north

In [ ]:
#import libraries
import pandas as pd
import numpy as np
from numpy import percentile
from numpy import unique
from numpy import where
import matplotlib as mpl
from matplotlib import pyplot
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns; sns.set(font_scale=1.2) 
from sklearn.ensemble import IsolationForest
from sklearn import metrics
import pyod 
from pyod.models.cblof import CBLOF
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from sklearn.mixture import GaussianMixture as GMM
from sklearn.cluster import KMeans
from sklearn.cluster import Birch
from sklearn.cluster import MiniBatchKMeans
from sklearn import metrics

# Data LOADING,CLEANING, FORMATING & VISUALIZATION

In [ ]:
#load Data

dataSUBSETan=pd.read_csv("C:/599_Research/FINAL_RESEARCH_and_PPT/THESIS_SUBMISSION/APPENDIX/3_MULTIPLE_ATTRIBUTE_SCRIPT/DATA/Merge_NORTH_A_SUBSET.csv")

dataSUBSETan

In [ ]:
#If dataset is covering many tiles we  can combine csv into 1 - we do this for ascending and descending and north and south
#df_asc_south = pd.concat([data1as, data2as, data3as, data4as, data5as, data6as], ignore_index=True)


In [ ]:
#inspect - dataSUBSETan
#dataSUBSETan.head()
dataSUBSETan.describe()
#dataSUBSETan.columns

In [ ]:
# Manually set CRS (it might work without depending on machine, but just in case)
dataSUBSETan.crs = {'init': u'epsg:4326'}

dataSUBSETan.info()

In [ ]:
#Visualize the data for the VEL(OR ANY OTHER ATTRIBUTE), changing the hue allows you to visualize any attribute
sns.set(style="whitegrid")
plt.scatter(dataSUBSETan['LONG'],dataSUBSETan['LAT'], c= dataSUBSETan['VEL'], s=1)

In [ ]:
#Get rid of any columns of information that are not needed ex below.(this saves on processing time)
#dataSUBSETan.drop(['D20191213','D20200126'], axis = 1, inplace = True)

# Estimate correct number of Clusters to Begin Looking for

In [ ]:
###specific to GMM!
#GMM
#the Akaike information criterion (AIC) or the Bayesian information criterion (BIC).
X = np.array(list(zip(dataSUBSETan['VEL'],dataSUBSETan['V_STDEV'])))
n_components = np.arange(1, 21)
models = [GMM(n, covariance_type='full', random_state=0).fit(X) for n in n_components]
plt.plot(n_components, [m.bic(X) for m in models], label='BIC')
plt.plot(n_components, [m.aic(X) for m in models], label='AIC')
plt.legend(loc='best')
plt.xlabel('n_components');

In [ ]:
def SelBest(arr:list, X:int)->list:
    '''
    returns the set of X configurations with shorter distance
    '''
    dx=np.argsort(arr)[:X]
    return arr[dx]

In [ ]:
#Silhouette Score
X = np.array(list(zip(dataSUBSETan['VEL'],dataSUBSETan['V_STDEV'])))
n_clusters=np.arange(2, 20)
sils=[]
sils_err=[]
iterations=20
for n in n_clusters:
    tmp_sil=[]
    for _ in range(iterations):
        gmm=GMM(n, n_init=2).fit(X) 
        labels=gmm.predict(X)
        sil=metrics.silhouette_score(X, labels, metric='euclidean')
        tmp_sil.append(sil)
    val=np.mean(SelBest(np.array(tmp_sil), int(iterations/5)))
    err=np.std(tmp_sil)
    sils.append(val)
    sils_err.append(err)
    
plt.errorbar(n_clusters, sils, yerr=sils_err)
plt.title("Silhouette Scores", fontsize=20)
plt.xticks(n_clusters)
plt.xlabel("N. of clusters")
plt.ylabel("Score")

In [ ]:
#gradient score
plt.errorbar(n_clusters, np.gradient(bics), yerr=bics_err, label='BIC')
plt.title("Gradient of BIC Scores", fontsize=20)
plt.xticks(n_clusters)
plt.xlabel("N. of clusters")
plt.ylabel("grad(BIC)")
plt.legend()

In [ ]:
#CREATE SINGLE ATTRIBUTE SUBPLOTS AND CORRELATION MATRIX TO FIND THE BEST COMBO OF ATTRIBUTES TO USE FOR MODEL ASSESSMENT
cluster = ['VEL','V_STDEV','ACC','COHERENCE','STD_DEF','SEASON_AMP','D20200224']
#SUBPLOTS
fig = plt.figure()

plt.rcParams['figure.figsize'] = [15,10]
plt.rcParams["font.weight"] = "bold"

fontdict={'fontsize': 25,
          'weight' : 'bold'}

fontdicty={'fontsize': 10,
          'weight' : 'bold',
          'verticalalignment': 'baseline',
          'horizontalalignment': 'center'}

fontdictx={'fontsize': 10,
          'weight' : 'bold',
          'horizontalalignment': 'center'}

plt.subplots_adjust(wspace=0.2, hspace=0.2)

fig.suptitle('Attribute Quick View', fontsize=25,fontweight="bold", color="black", 
             position=(0.5,1.01))

ax1 = fig.add_subplot(221)
ax1.scatter(dataSUBSETan['LONG'], dataSUBSETan['LAT'], c=dataSUBSETan['VEL'], s=10, cmap='viridis')
ax1.set_title('VEL', fontdict=fontdict, color="green")

ax2 = fig.add_subplot(222)
ax2.scatter(dataSUBSETan['LONG'], dataSUBSETan['LAT'], c=dataSUBSETan['ACC'], s=10, cmap='viridis')
ax2.set_title('ACC', fontdict=fontdict, color="orange")

ax3 = fig.add_subplot(223)
ax3.scatter(dataSUBSETan['LONG'], dataSUBSETan['LAT'], c=dataSUBSETan['COHERENCE'], s=10, cmap='viridis')
ax3.set_title('COHERENCE', fontdict=fontdict, color="brown")


ax4 = fig.add_subplot(224)
ax4.scatter(dataSUBSETan['LONG'], dataSUBSETan['LAT'], c=dataSUBSETan['V_STDEV'], s=10, cmap='viridis')
ax4.set_title("V_STDEV", fontdict=fontdict, color="blue")

#CORRELATION MATRIX
sns.pairplot(dataSUBSETan[cluster], kind='reg', diag_kind='kde')

In [ ]:
# define dataset
X = np.array(list(zip(dataSUBSETan['STD_DEF'], dataSUBSETan['VEL'],dataSUBSETan['ACC'],dataSUBSETan['COHERENCE'], dataSUBSETan['SEASON_AMP'])))
#df_asc_north['ACC'], df_asc_north['V_STDEV'], df_asc_north['STD_DEF'])))
# define the model & fit the model
kmeans_model = KMeans(n_clusters=5, random_state=1).fit(X)
# assign a cluster to each example
yhat = kmeans_model.predict(X)
# retrieve unique clusters
clusters = unique(yhat)
                  
import timeit

start = timeit.default_timer()

# All the program statements
stop = timeit.default_timer()
execution_time = stop - start

print("Program Executed in "+str(execution_time)) # It returns time in seconds

#map the labels to colors
c= ['b', 'r', 'y', 'g', 'c', 'm', 'e','f', 'u', 'd', 'a', 'h', 'i', 'j', 'k', 'l','n','o','p']
colors = [c[i] for i in yhat]

#Plot clusters with coordinates
figure(num=None, figsize=(10, 8), dpi=100, facecolor='w', edgecolor='k')
pyplot.scatter(dataSUBSETan['LONG'], dataSUBSETan['LAT'], c=yhat, s=3, cmap='viridis')
plt.savefig('dataSUBSETan_kmeans_PERMIAN_STD_DEF_VEL_ACC_COHERENCE_SEASON_AMP_5.png')

# 2. MODEL EXECUTION

In [ ]:
#kmeans
# define dataset
X = np.array(list(zip(dataSUBSETan['VEL'],dataSUBSETan['D20200224'],dataSUBSETan['ACC'],dataSUBSETan['SEASON_AMP'])))
# define the model & fit the model
kmeans_model = KMeans(n_clusters=5, random_state=1).fit(X)
# assign a cluster to each example
yhat = kmeans_model.predict(X)
# retrieve unique clusters
clusters = unique(yhat)

import timeit

start = timeit.default_timer()

# All the program statements
stop = timeit.default_timer()
execution_time = stop - start

print("Program Executed in "+str(execution_time)) # It returns time in seconds

#map the labels to colors
c= ['b', 'r', 'y', 'g', 'c', 'm', 'e','f', 'u', 'd', 'a', 'h', 'i', 'j', 'k', 'l','n','o','p']
colors = [c[i] for i in yhat]

#Plot clusters with coordinates
figure(num=None, figsize=(10, 8), dpi=100, facecolor='w', edgecolor='k')
pyplot.scatter(dataSUBSETan['LONG'], dataSUBSETan['LAT'], c=yhat, s=2, cmap='viridis')
plt.savefig('dataSUBSETan_kmeans_PERMIAN_all_no_QUALITY_5.png')

In [ ]:
#MiniBatch_Kmeans
# define dataset
X = np.array(list(zip(dataSUBSETan['VEL'],dataSUBSETan['D20200224'],dataSUBSETan['ACC'],dataSUBSETan['SEASON_AMP'])))
# fit the model
MiniBatch_model.fit(X)
# assign a cluster to each example
yhat = MiniBatch_model.predict(X)
# retrieve unique clusters
clusters = unique(yhat)


import timeit

start = timeit.default_timer()

# All the program statements
stop = timeit.default_timer()
execution_time = stop - start

print("Program Executed in "+str(execution_time)) # It returns time in seconds

#map the labels to colors
c= ['b', 'r', 'y', 'g', 'c', 'm', 'e','f', 'u', 'd', 'a', 'h']
colors = [c[i] for i in yhat]

#Plot clusters with coordinates
figure(num=None, figsize=(10, 8), dpi=100, facecolor='w', edgecolor='k')
pyplot.scatter(dataSUBSETan['LONG'], dataSUBSETan['LAT'], c=yhat, s=2, cmap='viridis')
plt.savefig('dataSUBSETan_MINIBATCH_PERMIAN_all_no_QUALITY_5.png')

In [ ]:
#GMM
# define dataset
X = np.array(list(zip(dataSUBSETan['VEL'],dataSUBSETan['D20200224'],dataSUBSETan['ACC'],dataSUBSETan['SEASON_AMP'])))
#define the model
GMM_model = GMM(n_components=5)
# fit the model
GMM_model.fit(X)
# assign a cluster to each example
yhat = GMM_model.predict(X)
# retrieve unique clusters
GMM_clusters = unique(yhat)

import timeit

start = timeit.default_timer()

# All the program statements
stop = timeit.default_timer()
execution_time = stop - start

print("Program Executed in "+str(execution_time)) # It returns time in seconds

#map the labels to colors
c= ['b', 'r', 'y', 'g', 'c', 'm', 'e','f', 'u', 'd', 'a', 'h']
colors = [c[i] for i in yhat]

#Plot clusters with coordinates
figure(num=None, figsize=(10, 8), dpi=100, facecolor='w', edgecolor='k')
pyplot.scatter(dataSUBSETan['LONG'], dataSUBSETan['LAT'], c=yhat, s=2, cmap='viridis')
plt.savefig('dataSUBSETan_GMM_PERMIAN_ALL_no_QUALITY_5.png')

In [ ]:
#BIRCH
X = np.array(list(zip(dataSUBSETan['VEL'],dataSUBSETan['D20200224'],dataSUBSETan['ACC'],dataSUBSETan['SEASON_AMP'])))
# define the model
Birch_model = Birch(threshold = 0.001, n_clusters=5)
# fit the model
Birch_model.fit(X)
# assign a cluster to each example
yhat = Birch_model.predict(X)
# retrieve unique clusters
clusters = unique(yhat)

import timeit

start = timeit.default_timer()

# All the program statements
stop = timeit.default_timer()
execution_time = stop - start

print("Program Executed in "+str(execution_time)) # It returns time in seconds

#map the labels to colors
c= ['b', 'r', 'y', 'g', 'c', 'm', 'e','f', 'u', 'd', 'a', 'h']
colors = [c[i] for i in yhat]

#Plot clusters with coordinates
figure(num=None, figsize=(10, 8), dpi=100, facecolor='w', edgecolor='k')
pyplot.scatter(dataSUBSETan['LONG'], dataSUBSETan['LAT'], c=yhat, s=2, cmap='viridis')
plt.savefig('dataSUBSETan_BIRCH_PERMIAN_ALL_no_QUALITY_5.png')

# 3. MODEL ASSESSMENT

In [ ]:
#Kmeans
# Number of clusters in labels, ignoring noise if present.
labels = kmeans_model.labels_
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

# create scatter plot for samples from each cluster
#for cluster in clusters:
	# get row indexes for samples with this cluster
	#row_ix = where(yhat == cluster)
	# create scatter of these samples
	#pyplot.scatter(X[row_ix, 0], X[row_ix, 1])
# show the plot
#pyplot.show()

#print("Silhouette Coefficient: %0.3f"
      #% metrics.silhouette_score(X, labels, metric='sqeuclidean'))
#Calinski-Harabasz Index
print("Calinski Harabasz Score: %0.3f"
      % metrics.calinski_harabasz_score(X, labels))
#Davies Bouldin Index
print("Davies Bouldin Index: %0.3f"
      % metrics.davies_bouldin_score(X, labels))

cluster_map = pd.DataFrame()
cluster_map['data_index'] = dataSUBSETan.index.values
cluster_map['cluster'] = kmeans_model.labels_

cluster_map[cluster_map.cluster == 4]

In [ ]:
#MINIBATCH kmeans
# Number of clusters in labels, ignoring noise if present.
labels = MiniBatch_model.labels_
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

# create scatter plot for samples from each cluster
#for cluster in clusters:
	# get row indexes for samples with this cluster
	#row_ix = where(yhat == cluster)
	# create scatter of these samples
	#pyplot.scatter(X[row_ix, 0], X[row_ix, 1])
# show the plot
#pyplot.show()

#print("Silhouette Coefficient: %0.3f"
      #% metrics.silhouette_score(X, labels, metric='sqeuclidean'))
#Calinski-Harabasz Index
print("Calinski Harabasz Score: %0.3f"
      % metrics.calinski_harabasz_score(X, labels))
#Davies Bouldin Index
print("Davies Bouldin Index: %0.3f"
      % metrics.davies_bouldin_score(X, labels))

cluster_map = pd.DataFrame()
cluster_map['data_index'] = dataSUBSETan.index.values
cluster_map['cluster'] = MiniBatch_model.labels_

cluster_map[cluster_map.cluster == 4]

In [ ]:
#GMM
GMM_model.score()
GMM_model.aic()



In [ ]:
#BIRCH
# Number of clusters in labels, ignoring noise if present.
labels = Birch_model.labels_
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

# create scatter plot for samples from each cluster
#for cluster in clusters:
	# get row indexes for samples with this cluster
	#row_ix = where(yhat == cluster)
	# create scatter of these samples
	#pyplot.scatter(X[row_ix, 0], X[row_ix, 1])
# show the plot
#pyplot.show()

#print("Silhouette Coefficient: %0.3f"
      #% metrics.silhouette_score(X, labels, metric='sqeuclidean'))
#Calinski-Harabasz Index
print("Calinski Harabasz Score: %0.3f"
      % metrics.calinski_harabasz_score(X, labels))
#Davies Bouldin Index
print("Davies Bouldin Index: %0.3f"
      % metrics.davies_bouldin_score(X, labels))

cluster_map = pd.DataFrame()
cluster_map['data_index'] = dataSUBSETan.values
cluster_map['cluster'] = Birch_model.labels_

cluster_map[cluster_map.cluster == 4]

# 4. EXPORT TO LAYER FOR ARCPRO: CSV TO SHP

In [ ]:
data.to_csv('C:/599_Research/ARTIFICIAL/Permian/SA_SHP_OUTPUTS/asc_south_kmeans_PERMIAN_VEL_6_withlabels.csv')

In [ ]:
# MakeXYLayer.py
# Description: Creates an XY layer and exports it to a layer file

# import system modules 
import arcpy
from arcpy import env

# Set environment settings
env.workspace = "C:/599_Research/ARTIFICIAL/Permian/SA_SHP_OUTPUTS"
 
# Set the local variables
in_Table = "asc_south_kmeans_PERMIAN_VEL_6_withlabels.csv"
x_coords = "LONG"
y_coords = "LAT"
z_coords = "HEIGHT"
out_Layer = "asc_south_kmeans_PERMIAN_VEL_6_withlabels_layer"
saved_Layer = r"C:/599_Research/ARTIFICIAL/Permian/SA_SHP_OUTPUTS/asc_south_kmeans_PERMIAN_VEL_6_MLOUTPUT.shp"
 
# Set the spatial reference
spRef = arcpy.SpatialReference(4326)
 
# Make the XY event layer...
arcpy.MakeXYEventLayer_management(in_Table, x_coords, y_coords, out_Layer, spRef, z_coords)
 
# Save to a layer file
arcpy.SaveToLayerFile_management(out_Layer, saved_Layer)